# yolo-preprocessed

In [1]:
%%capture
!pip install ultralytics wandb scikit-learn matplotlib
 

In [2]:
import os
from pathlib import Path
import shutil
import matplotlib.pyplot as plt
import numpy as np
from ultralytics import YOLO
import wandb
from sklearn.metrics import f1_score, confusion_matrix
import sys
sys.path.append('.')

In [3]:
run_name = "yolo11s-cls-pmg-512"
model_version = 'yolo11s-cls.pt'
data_dir = "./ODIR-2019/YOLO/processed_512g_merged"
project_name = "odir-2019-yolo"
saved_model_name = 'yolo11s-cls-odir-2019-pmg.pt'
result_folder = 'ODIR-2019/results/'
final_model_path = os.path.join(result_folder, saved_model_name)
image_size = 512
batch_size = 16
epochs = 1 # For demonstration, set to 1. Increase as needed to 50 or more.
workers_count = 4
patience=30

## Train

In [4]:
run_id=wandb.init(
    project=project_name,
    name=run_name,
    config={
        "img_size": image_size,
        "batch_size": batch_size,
        "epochs": epochs,
        "model": model_version,
        "patience": patience,
    },
    resume="allow"
)

wandb: Currently logged in as: raymond-samalo (samalo) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: WARNING Tried to log to step 1 that is less than the current step 2. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.


In [5]:
model = YOLO(model_version)

In [6]:
"""
Minimal code that WILL work for YOLOv11 classification F1 metrics.
Just copy, paste, and change your dataset path.
"""
from ultralytics import YOLO
import torch
from sklearn.metrics import f1_score, precision_score, recall_score
import numpy as np

# This callback WILL work at epoch end
def calculate_f1_at_epoch_end(trainer):
    """Simple, guaranteed-to-work epoch-end F1 calculation"""
    print(f"\n📊 Epoch {trainer.epoch} - Calculating F1 Score...")
    
    # Get validation data
    val_loader = trainer.validator.data_loader
    
    all_preds = []
    all_labels = []
    
    model = trainer.model
    model.eval()
    
    with torch.no_grad():
        for images, labels in val_loader:
            if torch.cuda.is_available():
                images = images.cuda()
            
            outputs = model(images)
            predictions = torch.argmax(outputs, dim=1)
            
            all_preds.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # Calculate F1
    f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)
    precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
    
    print(f"   ✅ F1: {f1:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f}")
    print(f"   ✅ Top-1 Acc: {np.mean(np.array(all_preds) == np.array(all_labels)):.4f}")
    
    return f1

In [7]:
model.add_callback("on_train_end", calculate_f1_at_epoch_end)
model.train(
    data=data_dir,        # folder-based (train/val)
    epochs=epochs,
    imgsz=image_size,
    batch=batch_size,
    device=0,
    workers=workers_count,
    patience=patience,
    pretrained=True,
    verbose=False)

New https://pypi.org/project/ultralytics/8.4.14 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.240 🚀 Python-3.12.12 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 2050, 3769MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./ODIR-2019/YOLO/processed_512g_merged, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train22, nbs=

AttributeError: 'ClassificationValidator' object has no attribute 'data_loader'

In [ ]:
wandb.finish()